# NOAA GFS Data Cube

This example loads a 3D atmospheric data cube from the [NOAA Global Forecast System (GFS)](https://www.ncei.noaa.gov/products/weather-climate-models/global-forecast) via the UCAR THREDDS Data Server using [Siphon](https://unidata.github.io/siphon/). The data has latitude, longitude, and isobaric (pressure level) dimensions, forming a true 3D volume.

We visualize temperature across all pressure levels, then extract isosurfaces (contours) to reveal 3D thermal structure.

> **Note:** This notebook requires an internet connection and the `siphon` package.

In [ ]:
import pyvista as pv
from siphon.catalog import TDSCatalog

import pvxarray  # noqa: F401

pv.set_plot_theme("document")
pv.set_jupyter_backend("server")  # Critical for large data

## Load GFS Data from THREDDS

Connect to UCAR's THREDDS catalog and open the GFS 0.25-degree analysis dataset remotely with xarray:

In [ ]:
catUrl = "https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg_ana/catalog.xml"
datasetName = "Full Collection Dataset"

catalog = TDSCatalog(catUrl)
ds = catalog.datasets[datasetName].remote_access(use_xarray=True)
ds

## Extract Temperature at One Timestep

Select the `Temperature_isobaric` variable at the first timestep. This gives us a 3D array: `(isobaric, lat, lon)`:

In [ ]:
da = ds["Temperature_isobaric"]
dt = da.isel(time=0)

mesh = dt.pyvista.mesh(x="lon", y="lat", z="isobaric")
mesh

## 3D Volume Rendering

Visualize the full temperature field. Vertical exaggeration is needed since pressure levels span a much smaller range than the horizontal coordinates:

In [ ]:
pl = pv.Plotter()
pl.add_mesh(mesh, cmap="coolwarm")
pl.set_scale(zscale=0.001)
pl.show(cpos="xy")

## Freezing Isosurface

Extract the 273.15 K (0 degrees C) isosurface to see where the freezing level sits in the atmosphere:

In [ ]:
pl = pv.Plotter()
pl.add_mesh(mesh.contour([273.15]), color=True)
pl.set_scale(zscale=0.001)
pl.show(cpos="xy")

## Multiple Temperature Contours

Extract multiple isosurfaces to reveal the full 3D thermal structure of the atmosphere:

In [ ]:
pl = pv.Plotter()
pl.add_mesh(mesh.contour(), cmap="coolwarm")
pl.set_scale(zscale=0.001)
pl.show(cpos="xy")